# Analyse des statistiques des axes bruts

Moyenne :
- Les moyennes des trois axes (-1.62, -0.88, -1.16) ne sont pas centrées autour de 0g ou 1g (gravité attendue pour un capteur correctement calibré).
- Cela indique un décalage constant (offset) dans vos données brutes.

Écart type (std) :
- Les écarts-types sont raisonnables (0.46g à 0.64g), mais le signal semble être affecté par des vibrations.

Valeurs extrêmes :
- Min : -9g pour les trois axes.
- Max : 6.98g pour deux axes et 4.65g pour un axe.

Ces valeurs semblent excessivement élevées, surtout si l’activité est limitée à une course ou une marche rapide. Cela peut indiquer du bruit mécanique ou des artefacts.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = "/Users/leasaab/Documents/Dev/AX3-Project/data/FemmeCardio.csv"

data = pd.read_csv(file_path)

timestamp_column = data.columns[0]
signal_columns = data.columns[1:]   # All remaining columns are sensor data

# Convert the timestamp column to datetime
data[timestamp_column] = pd.to_datetime(data[timestamp_column], errors='coerce')  # Handle invalid dates

# Calculate the sampling interval (in seconds)
sampling_interval = (
    pd.to_datetime(data[timestamp_column].iloc[1]) -  # Access a single row
    pd.to_datetime(data[timestamp_column].iloc[0])
).total_seconds()

# Create a relative time axis (in minutes)
data['relative_time_minutes'] = data.index * sampling_interval

# Define axis labels for clarity
axis_labels = ['X-axis', 'Y-axis', 'Z-axis']
colors = {'X': 'blue', 'Y': 'red', 'Z': 'green'}  # Define colors for each signal

In [3]:
print("Résumé des axes bruts (X, Y, Z) :")
print(data[signal_columns].describe())

Résumé des axes bruts (X, Y, Z) :
          -0.296875      0.703125     -0.343750
count  2.844599e+06  2.844599e+06  2.844599e+06
mean  -6.206059e-01  1.180590e-01 -1.640044e-01
std    6.417535e-01  5.719005e-01  4.655762e-01
min   -8.000000e+00 -8.000000e+00 -8.000000e+00
25%   -9.062500e-01 -2.187500e-01 -5.000000e-01
50%   -6.406250e-01  6.250000e-02 -2.343750e-01
75%   -3.125000e-01  5.781250e-01  1.562500e-01
max    7.984375e+00  7.984375e+00  5.656250e+00


• Problèmes identifiés

- Les données ne sont pas centrées correctement autour de la gravité (1g), affectant la norme euclidienne utilisée pour calculer l’ENMO.

- Les pics à -9g et 6.98g sont suspects et pourraient être causés par des bruits mécanique (vibrations du tapis roulant) ou une mauvaise calibration du capteur.

- L’absence de filtrage laisse passer les pics de bruit, amplifiant les anomalies dans l’ENMO.

Il faut donc corriger le décalage pour recentrer les moyennes autour de 1g pour l’axe vertical (Z), et 0g pour les autres axes. Puis appliquer un filtre passe-bas.

In [4]:
calibration_offset = {
    '-0.296875': -1.62,   # Offset pour l'axe X
    '0.703125': -0.88,    # Offset pour l'axe Y
    '-0.343750': -1.16    # Offset pour l'axe Z
}
for col, offset in calibration_offset.items():
    data[col] = data[col] - offset

## Additional infos

1. Sampling frequency

In [6]:
# Calcul de l'intervalle de temps entre les deux premiers échantillons
time_diff = (
    data[timestamp_column].iloc[1] - data[timestamp_column].iloc[0]
).total_seconds()

# Calcul de la fréquence d'échantillonnage (Hz)
fs = 1 / time_diff

print(f"La fréquence d'échantillonnage du signal est de {fs:.2f} Hz.")

La fréquence d'échantillonnage du signal est de 500.00 Hz.
